In [16]:
import pandas as pd
import numpy as np

In [17]:
read_pars = pd.read_csv('StockReorderlocations_2022-03-24.csv',header=0,dtype=str)
#print(read_pars.info())
read_pars.rename(columns={'ProductSKU':'SKU','MinimumBeforeReorder':'Par'},inplace=True)
read_pars.drop(['StockLocator','PickZones'],inplace=True,axis=1)
read_pars.Par = read_pars.Par.astype(float)
read_pars.Par = read_pars.Par.astype(int)
read_pars.ReorderQuantity = read_pars.ReorderQuantity.astype(float)
read_pars.ReorderQuantity = read_pars.ReorderQuantity.astype(int)
read_pars = read_pars.drop(columns=['Action','ReorderQuantity'])
read_pars.head()

,SKU,ProductName,Location,Par
0,086785024684,Drumshanbo Irish Gin 375ml,Oak Hill,1
1,086785024684,Drumshanbo Irish Gin 375ml,Pleasant Valley,4
2,086785024684,Drumshanbo Irish Gin 375ml,Airport,1
3,00850030152466,DEHY Grapefruit Jar,Pleasant Valley,2
4,00850030152466,DEHY Grapefruit Jar,Airport,2


In [18]:
df = pd.read_csv('AvailabilityReport_2022-03-24.csv',dtype=str)
#df.SKU = df.SKU.astype(int)
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18233 entries, 0 to 18232
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Category           18233 non-null  object
 1   SKU                18233 non-null  object
 2   ProductName        18233 non-null  object
 3   Location           18233 non-null  object
 4   Bin                0 non-null      object
 5   BatchSerialNumber  0 non-null      object
 6   ExpiryDate         0 non-null      object
 7   Discount           0 non-null      object
 8   StockValue         18233 non-null  object
 9   OnHand             18233 non-null  object
 10  VolumeOnHand       18233 non-null  object
 11  Available          18233 non-null  object
 12  OnOrder            18233 non-null  object
 13  InTransit          18233 non-null  object
 14  Allocated          18233 non-null  object
 15  Status             18233 non-null  object
 16  Unit               18233 non-null  objec

In [19]:
df = df.drop(columns=['Bin','BatchSerialNumber','ExpiryDate','Discount','StockValue','VolumeOnHand','Available','OnOrder','InTransit','Allocated','Status','Unit','Family','StockLocator','Brand','PriceTier1','PriceTier2','PriceTier3','NextDeliveryDate'])
df.OnHand = df.OnHand.astype(float)
df.OnHand = df.OnHand.astype(int)
df.head()

,Category,SKU,ProductName,Location,OnHand
0,Non Alcoholic Beverage,777823111113,1 Water 355ml,Airport,123
1,Non Alcoholic Beverage,777823111113,1 Water 355ml,Oak Hill,1
2,Non Alcoholic Beverage,777823111113,1 Water 355ml,Pleasant Valley,5
3,Bar Tools,1234500000000,12/24 Mixing Tin Sets,Airport,0
4,Bar Tools,1234500000000,12/24 Mixing Tin Sets,Pleasant Valley,0


In [20]:
ans = df.merge(right=read_pars,on=['SKU','ProductName','Location'])
ans.Par = ans.Par.astype(int)
ans.head()

,Category,SKU,ProductName,Location,OnHand,Par
0,Tequila,7503013136221,123 Organic Anejo Tequila 750ml,Warehouse,0,2
1,Tequila,7503013136207,123 Organic Blanco Tequila 750ml,Airport,0,0
2,Tequila,7503013136207,123 Organic Blanco Tequila 750ml,Oak Hill,1,0
3,Tequila,7503013136207,123 Organic Blanco Tequila 750ml,Pleasant Valley,1,1
4,Tequila,7503013136207,123 Organic Blanco Tequila 750ml,Warehouse,1,2


In [21]:
host_store = 'Oak Hill'

ans['Shy'] = ans.Par - ans.OnHand
#print(ans.head(10))
ans['Send'] = np.nan
for i, row in ans.iterrows():
    temp = ans[(ans.Location == host_store) & (ans.ProductName == row.ProductName)]
    if len(temp) > 0 and row.Shy > 0 and temp.iloc[0].Shy < 0:
        #print(i,'yes')
        if row.Shy <= (temp.iloc[0].Shy*-1):
            #print(i,'if')
            ans.at[i,'Send'] = row.Shy
            #row.iloc[i].['Send']='FUCK'
        else:
            #print(i,'else')
            ans.at[i,'Send'] = temp.iloc[0].Shy*-1
            #row['Send']='SHIT'


#ans['Candidate'] = ans.apply(checkstock,axis=1)

  Category            SKU                         ProductName  \
0  Tequila  7503013136221     123 Organic Anejo Tequila 750ml   
1  Tequila  7503013136207    123 Organic Blanco Tequila 750ml   
2  Tequila  7503013136207    123 Organic Blanco Tequila 750ml   
3  Tequila  7503013136207    123 Organic Blanco Tequila 750ml   
4  Tequila  7503013136207    123 Organic Blanco Tequila 750ml   
5  Tequila  7503013136214  123 Organic Reposado Tequila 750ml   
6  Tequila  7503013136214  123 Organic Reposado Tequila 750ml   
7  Tequila  7503013136214  123 Organic Reposado Tequila 750ml   
8  Tequila  7503013136214  123 Organic Reposado Tequila 750ml   
9  Bubbles   805300030616         17/24 Metodo Classico 750ml   

          Location  OnHand  Par  Shy  
0        Warehouse       0    2    2  
1          Airport       0    0    0  
2         Oak Hill       1    0   -1  
3  Pleasant Valley       1    1    0  
4        Warehouse       1    2    1  
5          Airport       0    0    0  
6         O

In [22]:
ans[ans.Send.isnull()==False].head(40)
ans.to_csv('InventoryShuffle.csv')

# Again but with *my* pars (shit this doesn't show warehouse)


In [23]:
reader = pd.read_csv('ParAnalysis.csv',dtype=str)
#print(reader.head())
my_pars = reader[['ProductSKU','Product','Customer','75']].copy()
my_pars.rename(columns={'ProductSKU':'SKU','Product':'ProductName','75':'Par','Customer':'Location'},inplace=True)
my_pars.Par = my_pars.Par.astype(float)
my_pars.Par = my_pars.Par.astype(int)
#my_pars.head()

In [24]:
list_of_retail_stores = ['Customer Square ' + str(x) for x in range(1,4)] #we have 3 stores active, we won't count the Warehouse
retail_names = ['Airport','Pleasant Valley','Oak Hill'] # make sure this matches the above list length - no known automation possible
store_map = dict(zip(list_of_retail_stores,retail_names))

my_pars['Location'] = my_pars.apply(lambda x: store_map[x.Location],axis=1 )
my_pars.head()


,SKU,ProductName,Location,Par
0,7503013136207,123 Organic Blanco Tequila 750ml,Airport,0
1,7503013136207,123 Organic Blanco Tequila 750ml,Pleasant Valley,0
2,7503013136207,123 Organic Blanco Tequila 750ml,Oak Hill,0
3,7503013136214,123 Organic Reposado Tequila 750ml,Airport,0
4,7503013136214,123 Organic Reposado Tequila 750ml,Pleasant Valley,0


In [25]:
#print(my_pars.info())
#print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9030 entries, 0 to 9029
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   SKU          9030 non-null   object
 1   ProductName  9030 non-null   object
 2   Location     9030 non-null   object
 3   Par          9030 non-null   int32 
dtypes: int32(1), object(3)
memory usage: 247.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18233 entries, 0 to 18232
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Category     18233 non-null  object
 1   SKU          18233 non-null  object
 2   ProductName  18233 non-null  object
 3   Location     18233 non-null  object
 4   OnHand       18233 non-null  int32 
dtypes: int32(1), object(4)
memory usage: 641.1+ KB
None


In [26]:
my = df.merge(right=my_pars,on=['SKU','ProductName','Location'])
#my = my.drop('Customer')
my.head()

,Category,SKU,ProductName,Location,OnHand,Par
0,Tequila,7503013136207,123 Organic Blanco Tequila 750ml,Airport,0,0
1,Tequila,7503013136207,123 Organic Blanco Tequila 750ml,Oak Hill,1,0
2,Tequila,7503013136207,123 Organic Blanco Tequila 750ml,Pleasant Valley,1,0
3,Tequila,7503013136214,123 Organic Reposado Tequila 750ml,Airport,0,0
4,Tequila,7503013136214,123 Organic Reposado Tequila 750ml,Oak Hill,1,0


In [27]:
host_store = 'Oak Hill'

my['Shy'] = my.Par - my.OnHand
my['Send'] = np.nan
for i, row in my.iterrows():
    temp = my[(my.Location == host_store) & (my.ProductName == row.ProductName)]
    if len(temp) > 0 and row.Shy > 0 and temp.iloc[0].Shy < 0:
        if row.Shy <= (temp.iloc[0].Shy*-1):
            my.at[i,'Send'] = row.Shy
        else:
            my.at[i,'Send'] = temp.iloc[0].Shy*-1

In [31]:
#host_store = 'Oak Hill'
#
#my['Shy'] = my.Par - my.OnHand
#my['Send'] = np.nan
#for row in my.itertuples():                          #############more efficient attempt
#    temp = my[(my.Location == host_store) & (my.ProductName == row.ProductName)]
#    if len(temp) > 0 and row.Shy > 0 and temp.iloc[0].Shy < 0:
#        if row.Shy <= (temp.iloc[0].Shy*-1):
#            my.at[i,'Send'] = row.Shy
#        else:
#            my.at[i,'Send'] = temp.iloc[0].Shy*-1

In [32]:
#my[my.Send.isnull()==False].head(40)
my.to_csv('InventoryShuffleMINE.csv',index=False)

In [29]:
bloat = my[(my.Location==host_store) & (my.ProductName.str.contains(' 50ml')==False)].sort_values(by='Shy',ascending=True)
bloat.to_csv('bloatOH.csv')